<a href="https://colab.research.google.com/github/zihadbappy/Autumn20labs/blob/master/Final_LSTM_BiLSTM_CNN%2BLSTM_CNN%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zihadbappy/bnfakedata.git

Cloning into 'bnfakedata'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [2]:
import pandas as pd
Dataset= pd.read_csv('/content/bnfakedata/bnfakefinal_no_duplicate.csv')

In [3]:
df = pd.DataFrame(Dataset)

In [4]:
df = pd.DataFrame(Dataset)


In [5]:
df=df.dropna()

In [6]:
df.groupby('label').agg('count')

,headline,content
label,,
0,2339,2339
1,2338,2338


In [7]:
X=df.content
Y=df.label

In [8]:
Y.value_counts()

0    2339
1    2338
Name: label, dtype: int64

In [9]:
X.shape

(4677,)

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping

Onehot Representation


In [11]:
voc_size=89000

In [12]:
onehot_repr=[one_hot(words,voc_size)for words in X]

In [13]:
len(onehot_repr)

4677

In [14]:
news_length=300
embedded_news=pad_sequences(onehot_repr,padding='pre',maxlen=news_length)
print(embedded_news)

[[    0     0     0 ... 57014 34240 76581]
 [69739 66752 75335 ... 62732 88095 54243]
 [    0     0     0 ... 29358 88095 54243]
 ...
 [    0     0     0 ... 81968 87387 29152]
 [    0     0     0 ... 65208 39825 26703]
 [    0     0     0 ... 79702  2085 32403]]


In [15]:
embedded_news[0]


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0, 84234, 54828, 78877,  7986, 30864, 71117,
       77791, 87958, 42630, 46706, 49077, 63324, 69739, 66752, 65488,
       84602, 65452, 54469,  1245, 50401, 65035, 33335, 34891, 53486,
       11170, 73761, 52356, 37623, 72600, 10733, 50628, 51896,  9296,
       55780, 24714,  6298, 46706, 49077, 63324, 69739, 66752, 75335,
       84602, 65452, 54469, 81545,  1245, 35976, 49396, 32666, 34990,
       51282, 46183,

In [16]:
len(embedded_news),Y.shape

(4677, (4677,))

In [17]:
import numpy as np
X_final=np.array(embedded_news)
y_final=np.array(Y)

In [18]:
X_final.shape,y_final.shape

((4677, 300), (4677,))

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=99)

# **LSTM MODEL**

In [20]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model1.add(LSTM(100))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 40)           3560000   
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 3,616,501
Trainable params: 3,616,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/8
55/55 [==============================] - 9s 30ms/step - loss: 0.4956 - accuracy: 0.7496 - val_loss: 0.2433 - val_accuracy: 0.9205
Epoch 2/8
55/55 [==============================] - 1s 21ms/step - loss: 0.1245 - accuracy: 0.9638 - val_loss: 0.1846 - val_accuracy: 0.9239
Epoch 3/8
55/55 [========

# **CNN+LSTM MODEL**

In [21]:
from keras.layers import Dense,Conv1D,MaxPooling1D

In [22]:
embedding_vecor_length = 40
model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 40)           3560000   
                                                                 
 conv1d (Conv1D)             (None, 300, 32)           3872      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 150, 32)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 3,617,173
Trainable params: 3,617,173
Non-trainable params: 0
____________________________________________

LSTM and CNN+LSTM evaluation

In [23]:
y_pred1=model1.predict(X_test)
y_pred2=model2.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [25]:
print("LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred1)))
print("CNN+LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred2)))

LSTM accuracy:0.9385
CNN+LSTM accuracy:0.9479


In [26]:
print("LSTM:\n", classification_report(y_test,np.around(y_pred1),digits=4))
print("\nCNN+LSTM:\n", classification_report(y_test,np.around(y_pred2),digits=4))

LSTM:
               precision    recall  f1-score   support

           0     0.9501    0.9237    0.9367       577
           1     0.9278    0.9528    0.9401       593

    accuracy                         0.9385      1170
   macro avg     0.9389    0.9383    0.9384      1170
weighted avg     0.9388    0.9385    0.9384      1170


CNN+LSTM:
               precision    recall  f1-score   support

           0     0.9708    0.9220    0.9458       577
           1     0.9277    0.9730    0.9498       593

    accuracy                         0.9479      1170
   macro avg     0.9492    0.9475    0.9478      1170
weighted avg     0.9489    0.9479    0.9478      1170



In [27]:
print("LSTM confusion matrix:\n ",confusion_matrix(y_test,np.around(y_pred1)))
print("\nCNN+LSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred2)))

LSTM confusion matrix:
  [[533  44]
 [ 28 565]]

CNN+LSTM confusion matrix:
 [[532  45]
 [ 16 577]]


#**Bi-LSTM MODEL**

In [28]:
embedding_vector_features=40
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model3.add(Bidirectional(LSTM(100)))
model3.add(Dropout(0.3))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 40)           3560000   
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 3,673,001
Trainable params: 3,673,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/8
55/55 [==============================] - 6s 51ms/step - loss: 0.6394 - accuracy: 0.7294 - val_l

## **CNN+BiLSTM MODEL**

In [29]:
embedding_vecor_length = 40
model4 = Sequential()
model4.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model4.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model4.add(MaxPooling1D(pool_size=2))
model4.add(Bidirectional(LSTM(100)))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model4.summary())
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 40)           3560000   
                                                                 
 conv1d_1 (Conv1D)           (None, 300, 32)           3872      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 150, 32)          0         
 1D)                                                             
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              106400    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 3,670,473
Trainable params: 3,670,473
No

In [30]:
y_pred3=model3.predict(X_test)
y_pred4=model4.predict(X_test)

In [31]:
print("BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred3)))
print("CNN+BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred4)))

BiLSTM accuracy:0.9453
CNN+BiLSTM accuracy:0.9496


In [32]:
print("BiLSTM:\n",classification_report(y_test,np.around(y_pred3),digits=4))
print("\nCNN+BiLSTM:\n",classification_report(y_test,np.around(y_pred4),digits=4))

BiLSTM:
               precision    recall  f1-score   support

           0     0.9476    0.9411    0.9443       577
           1     0.9430    0.9494    0.9462       593

    accuracy                         0.9453      1170
   macro avg     0.9453    0.9452    0.9453      1170
weighted avg     0.9453    0.9453    0.9453      1170


CNN+BiLSTM:
               precision    recall  f1-score   support

           0     0.9761    0.9203    0.9474       577
           1     0.9265    0.9781    0.9516       593

    accuracy                         0.9496      1170
   macro avg     0.9513    0.9492    0.9495      1170
weighted avg     0.9510    0.9496    0.9495      1170



In [33]:
print("BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred3)))
print("\nCNN+BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred4)))

BiLSTM confusion matrix:
 [[543  34]
 [ 30 563]]

CNN+BiLSTM confusion matrix:
 [[531  46]
 [ 13 580]]
